# Stages of Customer's Enterprise Architecture

This notebook contains stage diagrams for the first non-technology company we encountered that moved seemingly through all four stages of AI-Native transformation,
 entirely on their own only using ASE and a handful of other peers as architectural guides, subject-matter experts, and the sounding board.
 The engineering team at this company surpassed initial expectations from 2018 digital transformation program.
 The business software teams continued to grow, almost doubling in size, evolve, and iterate over business domain models.
 The documentation practices of this team are exemplary: architecture with code, architecture as code, define through tests and socialize.
 The most interesting evolution is that the team is using both the modular monolithic structure they were left with,
  and some of the most modern elements of distributed system architecture like microservices and microfrontends in amalgamation.

In my professional opinion this team surpasses Expedia Group innovation culture despite having shared no OSS components.

Here we celebrate their achievements by documenting their architectural decisions and practices.

In [ ]:
# Importing shared diagramming libraries
from urllib.request import urlretrieve

from diagrams import Diagram, Cluster, Edge
from diagrams.aws.management import ManagedServices

from diagrams.custom import Custom
from diagrams.generic.os import LinuxGeneral
from diagrams.onprem.ci import Jenkins
from diagrams.onprem.client import Users, Client

from diagrams.onprem.database import Oracle, MongoDB, Cassandra, Couchbase
from diagrams.onprem.etl import Embulk

from diagrams.generic.place import Datacenter
from diagrams.generic.storage import Storage
from diagrams.generic.compute import Rack

from diagrams.generic.network import Switch
from diagrams.onprem.identity import Dex
from diagrams.onprem.inmemory import Redis
from diagrams.onprem.logging import Fluentbit
from diagrams.onprem.network import Nginx
from diagrams.onprem.queue import Kafka
from diagrams.onprem.vcs import Git
from diagrams.openstack.sharedservices import Keystone
from diagrams.openstack.storage import Cinder, Manila, Swift

all_output_formats = ["png", "jpg", "svg", "pdf", "dot"]
output_format = "png"

dot_graph_attributes = {
    "fontsize": "32",
    "bgcolor": "transparent"
}

revolving_gate_url = "https://github.com/mingrammer/diagrams/raw/master/assets/img/diagrams.png"
revolving_gate_icon = "diagrams.png"
urlretrieve(revolving_gate_url, revolving_gate_icon)

openstack_url = "https://github.com/mingrammer/diagrams/raw/master/resources/openstack/openstack.png"
openstack_icon = "openstack.png"
urlretrieve(openstack_url, openstack_icon)

elastic_url = "https://github.com/mingrammer/diagrams/raw/master/resources/elastic/saas/elastic.png"
elastic_icon = "elastic.png"
urlretrieve(elastic_url, elastic_icon)

policy_url = "file:../assets/images/ase_tx_logo.png"
policy_icon = "ase_tx_logo.png"
urlretrieve(policy_url, policy_icon)


In [ ]:
with Diagram("2019 Modular Monolith",
             show=False,
             outformat=output_format,
             graph_attr=dot_graph_attributes,
             direction="TB",
             filename="origami-2019-modular-monolith") as complete_2019_picture:
    ctrx = Datacenter("Cloud\nContract")
    cann = Datacenter("Canonical\nEventstream\nDatacenter")
    bizz = Datacenter("Business\nDatacenter(s)")
    prod = Datacenter("Production\nDatacenter")

    with Cluster("Revolving\nDoor\nSecurity\nGateway"):
        root_policy = Custom("Trunk\nPolicy", policy_icon)
        policy_svc = ManagedServices("Managed\nServices")
        policy_relation = ctrx >> root_policy << policy_svc
        policy_broker = Custom("Revolving\nGateway\n(BROKER)", revolving_gate_icon)

    with Cluster("Root Domain\n(Eventstream Gateway)\n(Chinese Walls)"):
        eventstream_openstack = Custom("Public\nContract", openstack_icon)
        eventstream_replay_log = Fluentbit("Transaction\nReplay\nLog")
        eventstream_canonical_kafka = Kafka("Canonical\nEventstream\nKafka")
        canonical_relation = cann >> eventstream_openstack << eventstream_replay_log

        canonical_trunk = Git("Trunk")
        canonical_pipeline = Jenkins("Pipelines")

        eventstream_revolving_gate = Custom("Root\nRevolving\nGateway\n(AUTH)", revolving_gate_icon)
        canonical_gateway = Nginx("Canonical\nIngress")
        canonical_publishing = eventstream_canonical_kafka >> canonical_gateway
        canonical_vcs = canonical_pipeline >> canonical_trunk >> canonical_gateway

    with Cluster("Business Domain(s)\n(Operations 1-N)", direction="TB"):
        business_openstack = Custom("Public\nContract\nClient", openstack_icon)
        business_replay_log = Fluentbit("Domain\nTransaction\nReplay\nLog")
        business_canonical_kafka = Kafka("Canonical\nEventstream\nReplica\nKafka\nActive-Active")
        bounded_context_kafka = [Kafka("Bounded\nContext\n1"), Kafka("Bounded\nContext\nN")]
        domain_relation =  business_canonical_kafka >> business_replay_log >> business_openstack << bizz
        domain_event = bounded_context_kafka >> business_canonical_kafka

        business_revolving_gate = Custom("Revolving\nGateway\nPeer", revolving_gate_icon)
        bounded_context_ingress = Nginx("Domain\nIngress")
        canonical_business_mirroring = business_canonical_kafka >> bounded_context_ingress

    with Cluster("Production\n(Assembly 1-N)\nINACCESSIBLE", graph_attr={"nodesep": "0", "ranksep": "0"}):
        production_tech = LinuxGeneral("Embedded\nSystems")

        with Cluster("Robotic\nAssembly\nLine"):
            robotic_edges = [Rack("Edge 1"),
                             Rack("Edge N")]
        production_relation = robotic_edges >> production_tech << prod

        production_revolving_gate = Custom("Revolving\nGateway\nClient\n(READ)", revolving_gate_icon)
        # noinspection PyTypeChecker
        secured_synchronisation = business_revolving_gate - Edge(label="Revolving\nTTL-READ", style="dotted", color="green", fontcolor="green") << production_revolving_gate
        hidden_positioning = production_revolving_gate - Edge(color="black", style="invisible") - robotic_edges

    # noinspection PyTypeChecker
    revolver_cardinality = eventstream_revolving_gate >> Edge(label="Revolving\nAUTH/AUTH", style="bold", color="orange", fontcolor="orange") >> policy_broker
    # noinspection PyTypeChecker
    revolver_peer_cardinality = policy_broker << Edge(label="Revolving\nREAD/WRITE/SYNCH", style="bold", color="red", fontcolor="red") >> business_revolving_gate

    users = Users("Customer\nUsers")
    clients = Client("Client\nApplications")
    user_access = canonical_gateway - Edge(color="blue", style="bold") << users
    apps_access = canonical_gateway - Edge(color="blue", style="dashed") << clients
    traffic_cardinality = canonical_gateway - Edge(color="blue", style="bold") << bounded_context_ingress

complete_2019_picture